# Mellea Complete Tutorial

Welcome to the comprehensive interactive tutorial for **Mellea**.
This notebook combines practical usage patterns with core architectural concepts.

**Docs Reference:** [Welcome](https://docs.mellea.ai/overview/mellea-welcome), [Quickstart](https://docs.mellea.ai/overview/quick-start)

In [ ]:
import mellea
from mellea.backends import ModelOption
from mellea.backends.ollama import OllamaModelBackend

# 1. Initialize Mellea with the local Ollama backend and Granite model
# We use a fixed seed for reproducibility in this tutorial.
print("Initializing Mellea Session...")
m = mellea.MelleaSession(
    backend=OllamaModelBackend(
        model_id="granite4:micro", 
        model_options={ModelOption.SEED: 42} # Fixed seed for reproducibility
    )
)
print("Session started! Model: granite4:micro")

## Part 1: Basic Instruction

The core of Mellea is the `instruct` method. You give it a natural language instruction, and it returns a result.

In [ ]:
response = m.instruct("Write a short haiku about writing python code.")
print(response)

## Part 2: Components & CBlocks (Core Concepts)

Most LLM libraries just pass strings around. Mellea is different: it uses **Components**.

A `Component` is a structured object Mellea can format for an LLM call.

The minimum you need to define a custom component is:
- `parts()` → list of sub-parts (other Components or CBlocks)
- `format_for_llm()` → string (or TemplateRepresentation)

Then you can run it with `m.act(component)`.

In [ ]:
from mellea.core import Component, CBlock
import pydantic

# Define a structured output format using Pydantic
class FlashcardSetOut(pydantic.BaseModel):
    topic: str
    cards: list[dict]

# Define a custom Component
class FlashcardMaker(Component):
    def __init__(self, topic: str, count: int = 5):
        self.topic = topic
        self.count = count

    def parts(self):
        # We don't have nested parts in this simple example.
        return []

    def format_for_llm(self):
        # Returning a STRING is the simplest valid way to represent a Component.
        return (
            "You are a study assistant.\n"
            f"Make {self.count} flashcards about: {self.topic}.\n"
            "Return JSON with keys: topic (string), cards (list of objects).\n"
            "Each card object must have: question, answer, difficulty.\n"
        )

    def _parse(self, computed):
        # In a real component, you might parse the JSON here.
        # For this tutorial, we return the raw value and parse it manually outside.
        return computed.value

# Act on the component
print("Generating Flashcards...")
res = m.act(FlashcardMaker("OOP in Python", count=3), format=FlashcardSetOut)

print("\nRaw model output:", res.value)

# Parse it back into the Pydantic model
parsed = FlashcardSetOut.model_validate_json(res.value)
print("\nParsed Object:", parsed)

# The Loop: Instruct-Validate-Repair (Docs: Requirements)

Generative models can be unpredictable. Mellea's power lies in its **Instruct-Validate-Repair** loop. 
You can define **Requirements** (`req`) and **Checks** (`check`) that the output *must* satisfy. If the model fails, Mellea uses a **Strategy** to try again or fix it.

**Docs Reference:** [Requirements](https://docs.mellea.ai/overview/requirements)

### Example: Generating a Helper Email with Constraints
We want to generate an email that:
1.  Has a salutation.
2.  Does **not** use the word 'regards'.
3.  (Optional strict check) Uses only lower-case letters (just to show validation failure and repair).

In [ ]:
from mellea.stdlib.sampling import RejectionSamplingStrategy
from mellea.stdlib.requirements import req,check,simple_validate

# using a validation fn to validate your results as well as a check to validate your results
requirements = [
    req("The email should have a salutation"),
    req("Use only lower-case letters", validation_fn=simple_validate(lambda s: s.islower())),
    check("The email should not mention the word 'regards'")
        
]

def write_email(m:mellea.MelleaSession, names :str , notes:str)->str:
    email_candidate = m.instruct(
        "Write an email to {{name}} using the notes following: {{notes}}.",
        requirements=requirements,
        strategy = RejectionSamplingStrategy(loop_budget = 5),
        user_variables={"name": names, "notes": notes},
        return_sampling_results = True ,
    )
    if email_candidate.success :
        return str(email_candidate.result)
    else :
        print("Expect sub-par output due to failure to meet requirements.")
        return email_candidate.sampling_results[0].value
    
m=mellea.start_session()
print(write_email(m,"Arush","Arush has been a great team player, always willing to help others and contribute to group projects."))


# Advanced: Core Concepts & Generative Programming

**Docs Reference:** [Generative Programming](https://docs.mellea.ai/overview/project-mellea), [Core Concepts](https://docs.mellea.ai/core-concept/generative-slots)

In this section, we touch on more advanced concepts. Mellea treats prompts not just as strings, but as programs with **Slots**, **Context**, and **Agents**.

### Variable Injection (Context)
You saw `user_variables` above. This is part of Mellea's Context Management system, allowing you to inject data safely into prompts.

In [ ]:
# Quick example of dynamic variables
response = m.instruct(
    "Translate the following word to Spanish: {{word}}",
    user_variables={"word": "Hello"}
)
print(f"Hello in Spanish: {response}")

In [ ]:
# using model options
import mellea
from mellea.backends import ModelOption  #its mellea.backends
from mellea.backends.ollama import OllamaModelBackend
from mellea.backends import model_ids

m = mellea.MelleaSession(
    backend=OllamaModelBackend(
        model_id="granite4:micro",model_options={ModelOption.SEED: 42}
    )
)

answer = m.instruct(
    "What is 2x2?",
    model_options={
        "temperature": 0.1,
    },
)

print(str(answer))

## Generative Slots

``GenerativeSlot`` is a function whose implementation is provieded by an LLM.In Mellea you define these using ``@generative`` decorator

In [ ]:
# from mellea.std.base import ChatContext is old 
from mellea.stdlib.context import ChatContext
from mellea import start_session
from mellea.backends import ModelOption
from mellea import generative   

# ChatContext is used to maintain conversation history across multiple model calls. 
# Unlike SimpleContext (the default), which resets the chat history on each call, 
# ChatContext behaves like a chat history where previous messages are remembered.

@generative
def grade_syntax(code: str) -> int:
    """ Grade the code based on correct implementation of the function
    args:
        code: str (to be graded)
    returns:
        int : a grade between 1(worst) and 10(best)
    """
codes = (
    " def add(a, b):\n    return a + b",
    "def subtract(a,b): cout<<a-b<<endl",
    "int multiply(int a,int b) { return a*b}"
)
m = start_session(ctx = ChatContext() , model_options = {ModelOption.MAX_NEW_TOKENS: 10})
for code in codes :
    grade = grade_syntax(m, code = code)
    print("grade = ",grade)

### Advanced: Using Generative slots to Provide Compositionality Across Module Boundaries

In [ ]:
# file: https://github.com/generative-computing/mellea/blob/main/docs/examples/tutorial/compositionality_with_generative_slots.py#L1-L18
from mellea import generative
from typing import Literal

# The Summarizer Library
@generative
def summarize_meeting(transcript: str) -> str:
  """Summarize the meeting transcript into a concise paragraph of main points."""

@generative
def summarize_contract(contract_text: str) -> str:
  """Produce a natural language summary of contract obligations and risks."""

@generative
def summarize_short_story(story: str) -> str:
  """Summarize a short story, with one paragraph on plot and one paragraph on broad themes."""


# The Decision Aides Library
@generative
def propose_business_decision(summary: str) -> str:
  """Given a structured summary with clear recommendations, propose a business decision."""

@generative
def generate_risk_mitigation(summary: str) -> str:
  """If the summary contains risk elements, propose mitigation strategies."""

@generative
def generate_novel_recommendations(summary: str) -> str:
  """Provide a list of novel recommendations that are similar in plot or theme to the short story summary."""

# Compose the libraries.
@generative
def has_structured_conclusion(summary: str) -> Literal["yes", "no"]:
  """Determine whether the summary contains a clearly marked conclusion or recommendation."""

@generative
def contains_actionable_risks(summary: str) -> Literal["yes", "no"]:
  """Check whether the summary contains references to business risks or exposure."""

@generative
def has_theme_and_plot(summary: str) -> Literal["yes", "no"]:
  """Check whether the summary contains both a plot and thematic elements."""


In [ ]:
m = start_session()
transcript = """Meeting Transcript: Market Risk Review -- Self-Sealing Stembolts Division
Date: December 1, 3125
Attendees:

Karen Rojas, VP of Product Strategy

Derek Madsen, Director of Global Procurement

Felicia Zheng, Head of Market Research

Tom Vega, CFO

Luis Tran, Engineering Liaison

Karen Rojas:
Thanks, everyone, for making time on short notice. As you've all seen, we've got three converging market risks we need to address: tariffs on micro-carburetors, increased adoption of the self-interlocking leafscrew, and, believe it or not, the "hipsterfication" of the construction industry. I need all on deck and let's not waste time. Derek, start.

Derek Madsen:
Right. As of Monday, the 25% tariff on micro-carburetors sourced from the Pan-Alpha Centauri confederacy is active. We tried to pre-purchase a three-month buffer, but after that, our unit cost rises by $1.72. That's a 9% increase in the BOM cost of our core model 440 stembolt. Unless we find alternative suppliers or pass on the cost, we're eating into our already narrow margin.

Tom Vega:
We cannot absorb that without consequences. If we pass the cost downstream, we risk losing key mid-tier OEM clients. And with the market already sniffing around leafscrew alternatives, this makes us more vulnerable.

Karen:
Lets pause there. Felicia, give us the quick-and-dirty on the leafscrew.

Felicia Zheng:
It's ugly. Sales of the self-interlocking leafscrew—particularly in modular and prefab construction—are up 38% year-over-year. It's not quite a full substitute for our self-sealing stembolts, but they are close enough in function that some contractors are making the switch. Their appeal? No micro-carburetors, lower unit complexity, and easier training for install crews. We estimate we've lost about 12% of our industrial segment to the switch in the last two quarters.

Karen:
Engineering, Luis; your take on how real that risk is?

Luis Tran:
Technically, leafscrews are not as robust under high-vibration loads. But here's the thing: most of the modular prefab sites don not need that level of tolerance. If the design spec calls for durability over 10 years, we win. But for projects looking to move fast and hit 5-year lifespans? The leafscrew wins on simplicity and cost.

Tom:
So they're eating into our low-end. That's our volume base.

Karen:
Exactly. Now let's talk about this last one: the “hipsterfication” of construction. Felicia?

Felicia:
So this is wild. We're seeing a cultural shift in boutique and residential construction—especially in markets like Beckley, West Sullivan, parts of Osborne County, where clients are requesting "authentic" manual fasteners. They want hand-sealed bolts, visible threads, even mismatched patinas. It's an aesthetic thing. Function is almost secondary. Our old manual-seal line from the 3180s? People are hunting them down on auction sites.

Tom:
Well, I'm glad I don't have to live in the big cities... nothing like this would ever happen in downt-to-earth places Brooklyn, Portland, or Austin.

Luis:
We literally got a request from a design-build firm in Keough asking if we had any bolts “pre-distressed.”

Karen:
Can we spin this?

Tom:
If we keep our vintage tooling and market it right, maybe. But that's niche. It won't offset losses in industrial and prefab.

Karen:
Not yet. But we may need to reframe it as a prestige line—low volume, high margin. Okay, action items. Derek, map alternative micro-carburetor sources. Felicia, get me a forecast on leafscrew erosion by sector. Luis, feasibility of reviving manual seal production. Tom, let's scenario-plan cost pass-through vs. feature-based differentiation.

Let's reconvene next week with hard numbers. Thanks, all."""
summary = summarize_meeting(m, transcript=transcript)

if contains_actionable_risks(m, summary=summary) == "yes":
    mitigation = generate_risk_mitigation(m, summary=summary)
    print(f"Mitigation: {mitigation}")
else:
    print("Summary does not contain actionable risks.")
if has_structured_conclusion(m, summary=summary) == "yes":
    decision = propose_business_decision(m, summary=summary)
    print(f"Decision: {decision}")
else:
    print("Summary lacks a structured conclusion.")

### Mify Objects

```MObject``` storing data along side with its relevant operations(tools).This allows LLMs to interact with both tthe data and methods in a unifiied structured manner , simplifies exposinhg only specific fields and methods for LLMs to interact with.

```Mobject``` pattern also provides a way of evolvoing exsisting classical codebases into generative programs.

In [ ]:
import mellea
# old was mellea.stdlib.mify 
from mellea.stdlib.components.mify import mify, MifiedProtocol
import pandas as pd
from io import StringIO

@mify(fields_include = {"table"} , template = "{{table}}")
class myDB:
    table : str = """| Store      | Sales   |
                    | ---------- | ------- |
                    | Northeast  | $250    |
                    | Southeast  | $80     |
                    | Midwest    | $420    |"""

    def transpose(self):
        pd.read_csv(
            StringIO(self.table) , 
            sep = "|" ,
            skipinitialspace= True ,
            header = 0 ,
            index_col = False 
        )

m = mellea.start_session()
db = myDB()
assert isinstance(db , MifiedProtocol)
answer = m.query( db , " What is the total sales of the east ?")
print(str(answer))

### How to find new updates in the code base that cause you to use older imports and throws errors ?
```
(mellea) ashoksharma@Ashoks-MacBook-Pro Mellea_IBM % cd mellea
(mellea) ashoksharma@Ashoks-MacBook-Pro mellea % find . -name "mify.py"
./mellea/stdlib/components/mify.py
./docs/examples/mify/mify.py 
```

In [ ]:
# Working with documents

from mellea.stdlib.components.docs.richdocument import RichDocument
# Note: This requires the docling package installed
# rd = RichDocument.from_document_file("https://arxiv.org/pdf/1906.04043")

In [ ]:
# Example of working with a table from a document
from mellea.stdlib.components.docs.richdocument import Table
# table1 : Table = rd.get_tables()[0] 
# print(table1.to_markdown())

### Debugging: Session Reset & Last Turn

These are the 3 things you should check *all the time* when learning:

1. `m.last_prompt()` → what did you actually send?
2. `m.ctx.last_turn()` → last user+assistant pair
3. `m.ctx.view_for_generation()` (advanced) → what context will be sent next

In [ ]:
from mellea.backends.types import ModelOption
from mellea import start_session

print("Initializing Mellea Session...")
m = mellea.MelleaSession(
    backend=OllamaModelBackend(
        model_id="granite4:micro", 
        model_options={ ModelOption.SEED: 42, ModelOption.MAX_NEW_TOKENS:500 } # Fixed seed for reproducibility
    )
)
print("Session started! Model: granite4:micro")

In [ ]:
# Debugging commands
print("--- last_prompt ---")
print(m.last_prompt())

print("\n--- last_turn ---")
print(m.ctx.last_turn() if hasattr(m, "ctx") else "no ctx")

## Building Agents

Mellea supports building agents like ReACT/Reasoning Loop agents. 
See `demo_agent.py` for a full implementation of a ReACT agent.

## Interoperability

- **MCP (Model Context Protocol)**: Expose Mellea tools to other AI clients. (See `demo_mcp.py`)
- **crewAI Integration**: Mellea can be used as a backend for crewAI agents.